In [44]:
from textblob import TextBlob
import pandas as pd
import numpy as np

In [45]:
df = pd.read_csv('./data/processed/tx.csv', lineterminator='\n')
pd.set_option('display.max_columns', None)

In [46]:
df.head()

,Unnamed: 0,coordinates,created_at,hashtags,media,urls,favorite_count,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,lang,place,possibly_sensitive,retweet_count,retweet_id,retweet_screen_name,source,text,tweet_url,user_created_at,user_screen_name,user_default_profile_image,user_description,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,user_location,user_name,user_screen_name.1,user_statuses_count,user_time_zone,user_urls,user_verified
0,24,"-97.7347,30.5108",Thu Jul 23 05:46:41 +0000 2020,madeintexas,NaN,https://www.instagram.com/p/CC-KzNJDKLT/?igshi...,0,1286175883309592576,NaN,NaN,NaN,en,"Brushy Creek, TX",False,0,NaN,NaN,"<a href=""http://instagram.com"" rel=""nofollow"">...",|• I ❤️ My Texas Market Box 📍@mytexasmarket 📍 ...,https://twitter.com/Foxactress/status/12861758...,Sun Mar 22 06:08:47 +0000 2009,Foxactress,False,📝Writer |🎭Actor |🇺🇸📍LA/AUSTIN yenisask@gmail.com,3879,4540,3372,43,Los Angeles/ Austin,Yenis Monterrey,Foxactress,4550,NaN,https://amzn.to/2r1kPiy,False
1,32,"-95.3694,29.7602",Thu Jul 23 04:50:53 +0000 2020,waterpuddle rainpuddle reflection shadow daily...,NaN,https://www.instagram.com/p/CC-MjFOJNvM/?igshi...,0,1286161841882832896,NaN,NaN,NaN,en,"Houston, TX",False,0,NaN,NaN,"<a href=""http://instagram.com"" rel=""nofollow"">...",After the rain. #waterpuddle #rainpuddle #refl...,https://twitter.com/LoveTheLibrary1/status/128...,Sat Nov 01 18:28:31 +0000 2014,LoveTheLibrary1,False,Literacy Advocate & Teacher-Librarian. Writer ...,13504,1304,2476,142,Houston TX,Lisa Sandoz Robinson,LoveTheLibrary1,14909,NaN,NaN,False
2,46,"-97.5019,25.9049",Thu Jul 23 04:52:37 +0000 2020,NaN,NaN,https://www.instagram.com/p/CC-Mv9TnWF2/?igshi...,0,1286162276031094784,NaN,NaN,NaN,en,"Brownsville, TX",False,0,NaN,NaN,"<a href=""http://instagram.com"" rel=""nofollow"">...",The city of Brownsville announced they are cur...,https://twitter.com/956News/status/12861622760...,Mon Jan 20 05:48:10 +0000 2020,956News,False,NaN,0,1,2,0,NaN,956 News,956News,44,NaN,NaN,False
3,202,"-96.73981,32.90747",Thu Jul 23 12:06:06 +0000 2020,NaN,NaN,https://www.instagram.com/p/CC--W93nAEA/?igshi...,0,1286271367915147264,NaN,NaN,NaN,en,"Dallas, TX",False,0,NaN,NaN,"<a href=""http://instagram.com"" rel=""nofollow"">...",It's been four months in relative captivity. C...,https://twitter.com/PositivelyfitLH/status/128...,Tue Oct 21 16:00:28 +0000 2014,PositivelyfitLH,False,"Positively Fit Lake Highlands, Mallory Cislo, ...",582,117,246,0,"Dallas, TX",Positively Fit LH,PositivelyfitLH,356,NaN,http://www.positivelyfitlakehighlands.com,False
4,253,"-98.62473918,29.57596474",Thu Jul 23 12:32:44 +0000 2020,NaN,NaN,https://www.instagram.com/p/CC_BZ7jHjgd/?igshi...,0,1286278071511707648,NaN,NaN,NaN,en,"San Antonio, TX",False,0,NaN,NaN,"<a href=""http://instagram.com"" rel=""nofollow"">...",Guess where we’re going next week?! We’re kick...,https://twitter.com/inBalanceSA/status/1286278...,Thu Oct 03 19:23:11 +0000 2013,inBalanceSA,False,⭐️Pilates based fitness for every BODY ⭐️found...,40,113,162,16,"San Antonio, TX",inBalance,inBalanceSA,800,NaN,http://inbalancestudios.com,False


#### combine hashtags and text

In [47]:
df['all_text'] = df['hashtags'] + ' ' + df['text']

#### calculate sentiment

In [48]:
df['TB_polarity'] = df['all_text'].map(lambda x: TextBlob(str(x)).sentiment.polarity)
df['TB_subjectivity'] = df['all_text'].map(lambda x: TextBlob(str(x)).sentiment.subjectivity)


#### remove rows without text

In [49]:
df = df[df['all_text'].notna()]

#### convert created_at to datetime object, set date as index, drop unnecessary columns

In [50]:
df['created_at'] = pd.to_datetime(df['created_at']).dt.date

df = df.set_index(df['created_at'])

df = df.loc[:, ['all_text', 'TB_polarity', 'TB_subjectivity']]

df.index = pd.to_datetime(df.index)

#### save file

In [51]:
df.to_hdf('./data/processed/tx_sentiment_all.hdf', key='df')

#### groupby dates, calculate mean sentiment per day

In [52]:
df = df.groupby('created_at').mean()

#### save file

In [53]:
df.to_hdf('./data/processed/tx_sentiment_mean.hdf', key='df')